In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 32, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
        
        self.lineartest0 = nn.Linear(2, emsize)
        self.lineartest1 = nn.Linear(2, emsize)
        self.lineartest2 = nn.Linear(2, emsize)
        self.lineartest3 = nn.Linear(2 * emsize, emsize)
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 2]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def test_encode(self, src): #src must be [batchsize * nsamples, npoints = 5, 2]
        point_1 = self.lineartest1(src[:, 3, :] - 0.5) #want mean 0!!!!
        point_2 = self.lineartest1(src[:, 4, :] - 0.5)
        remaining = self.lineartest0(src[:, :3, :]) #[batchsize * nsamples, 3, emsize]
        point_1_message = self.lineartest1(src[:, 3, :] - 0.5).squeeze(1)
        point_2_message = self.lineartest2(src[:, 4, :] - 0.5).squeeze(1)
        point_1 = F.relu(torch.cat([point_1, point_2_message], dim = 1))
        point_2 = F.relu(torch.cat([point_2, point_1_message], dim = 1))
        point_1 = self.lineartest3(point_1)
        point_2 = self.lineartest3(point_2)
        src = torch.cat([torch.zeros_like(remaining.transpose(0, 1)), point_1.unsqueeze(0), point_2.unsqueeze(0)])
        return src #[npoints = 5, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 3], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 3, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 3, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 3, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, dtype = torch.bool, device = device).triu(1) #[npoints - 3, npoints - 3], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters(), lr = 0.003)


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 300000, npoints = 13, batchsize = 2000, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        regular_reinforcement = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1)
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) <= -0.05 #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1)) > 1
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 1
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 6) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss with thresholding
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * regular_reinforcement * (positive_reinforcement_binary + negative_reinforcement_binary * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 40))).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item())#, env.logprob[0].item())
  

In [ ]:
train(300000, 53, 800, 8)

150.4824981689453 -147.64659118652344 -148.46755981445312 -147.66226196289062 -148.46722412109375
131.81625366210938 -140.64048767089844 -147.5859832763672 -140.46017456054688 -148.09500122070312
120.10250091552734 -146.10887145996094 -148.4088134765625 -145.72528076171875 -148.52964782714844
123.56999969482422 -145.41928100585938 -148.3621826171875 -145.25473022460938 -148.4396514892578
123.82999420166016 -143.00389099121094 -148.09996032714844 -143.06175231933594 -147.70150756835938
127.00875091552734 -138.28268432617188 -147.1606903076172 -137.23635864257812 -144.43531799316406
124.78874969482422 -141.3874053955078 -147.85011291503906 -141.11773681640625 -149.03994750976562
115.35250091552734 -144.64085388183594 -148.2943115234375 -144.76898193359375 -149.38540649414062
108.5374984741211 -143.4522705078125 -148.1907958984375 -144.3723907470703 -148.676513671875
105.125 -137.94171142578125 -147.07945251464844 -137.97671508789062 -148.04989624023438
104.95999908447266 -127.92820739746

107.84874725341797 -19.086278915405273 -31.65267562866211 -18.252117156982422 -36.86023712158203
107.69874572753906 -18.42156982421875 -30.504776000976562 -21.220705032348633 -36.42634582519531
107.08999633789062 -19.054882049560547 -30.921628952026367 -19.090755462646484 -43.533103942871094
106.05500030517578 -21.353525161743164 -33.85081481933594 -17.09926986694336 -30.84596824645996
105.9012451171875 -26.006637573242188 -40.465065002441406 -25.513744354248047 -37.855995178222656
106.87625122070312 -25.667287826538086 -39.413692474365234 -19.990665435791016 -25.969343185424805
106.33749389648438 -21.487045288085938 -34.21144104003906 -23.276599884033203 -27.473861694335938
106.83124542236328 -18.9580078125 -30.84636688232422 -16.833097457885742 -24.3242130279541
105.5374984741211 -21.37973403930664 -32.99919128417969 -20.325952529907227 -31.963674545288086
103.87749481201172 -29.78447151184082 -42.146793365478516 -30.906829833984375 -55.18940734863281
104.60124969482422 -48.371032714

104.22249603271484 -21.465377807617188 -32.201316833496094 -21.658096313476562 -31.237472534179688
104.1824951171875 -22.184782028198242 -32.870880126953125 -19.349502563476562 -38.00054931640625
104.47750091552734 -21.914419174194336 -32.473018646240234 -21.147296905517578 -22.241193771362305
104.03125 -23.37360382080078 -33.99745178222656 -17.219886779785156 -23.831029891967773
104.17874908447266 -23.153913497924805 -33.681671142578125 -22.92182159423828 -34.398406982421875
104.13874816894531 -19.566423416137695 -29.370128631591797 -27.53033447265625 -28.310382843017578
103.84624481201172 -18.117944717407227 -27.60226058959961 -16.04488754272461 -31.30014419555664
104.3187484741211 -17.57292938232422 -26.82373809814453 -17.077898025512695 -28.993732452392578
104.7874984741211 -18.658632278442383 -27.92669105529785 -19.36614990234375 -26.63532066345215
105.05500030517578 -18.6923770904541 -28.106067657470703 -21.707592010498047 -21.965023040771484
105.02874755859375 -19.37439155578613

102.47249603271484 -7.861209392547607 -14.05086612701416 -8.182567596435547 -18.36565399169922
102.46125030517578 -7.205838680267334 -13.057132720947266 -8.066356658935547 -16.76552963256836
103.14249420166016 -7.340661525726318 -13.172182083129883 -7.830342769622803 -12.33542251586914
103.29999542236328 -7.932648658752441 -13.90631103515625 -8.693282127380371 -8.655664443969727
103.09249877929688 -7.446855068206787 -13.200560569763184 -8.195022583007812 -12.346783638000488
102.89374542236328 -6.956880569458008 -12.483711242675781 -10.45196533203125 -11.61849308013916
103.0062484741211 -6.368834018707275 -11.766873359680176 -6.8126115798950195 -7.440239906311035
102.60749816894531 -6.09941291809082 -11.432328224182129 -3.1492807865142822 -11.262208938598633
102.92749786376953 -6.0655670166015625 -11.448567390441895 -6.751439094543457 -13.224187850952148
102.62249755859375 -6.246331691741943 -11.645134925842285 -6.08968448638916 -6.51252555847168
102.72249603271484 -6.3166399002075195 -

102.54750061035156 -4.912810325622559 -9.276655197143555 -5.854252338409424 -9.334650039672852
102.97249603271484 -4.825686931610107 -9.203980445861816 -5.295110702514648 -18.67568588256836
102.91749572753906 -4.66060209274292 -8.868701934814453 -4.573697090148926 -6.7889580726623535
102.69124603271484 -4.6294169425964355 -8.839394569396973 -5.360390663146973 -8.021859169006348
102.52749633789062 -4.505586624145508 -8.630663871765137 -4.459978103637695 -10.963630676269531
102.4574966430664 -4.535960674285889 -8.54354190826416 -3.6892170906066895 -9.891298294067383
103.22874450683594 -4.41310977935791 -8.387782096862793 -4.929698944091797 -6.953034400939941
102.71749877929688 -4.442592620849609 -8.455362319946289 -2.889547348022461 -13.780458450317383
102.7249984741211 -4.320319652557373 -8.200716018676758 -3.4624061584472656 -5.298858165740967
102.6199951171875 -4.214584827423096 -8.05195426940918 -3.203944683074951 -7.147912979125977
102.38499450683594 -4.224513530731201 -8.0180463790

...

112.08625030517578 -148.34527587890625 -148.35403442382812 -148.34375 -148.15625
113.0374984741211 -148.17312622070312 -148.52499389648438 -148.46875 -148.8125
112.38249969482422 -148.26785278320312 -148.47988891601562 -148.59375 -148.15625
112.32374572753906 -148.51617431640625 -148.52259826660156 -148.65625 -148.46875
111.95249938964844 -148.25588989257812 -148.42678833007812 -148.15625 -148.53125
112.13874816894531 -148.33370971679688 -148.34408569335938 -148.59375 -148.28125
112.31499481201172 -148.38504028320312 -148.40664672851562 -148.46875 -148.21875
112.14875030517578 -148.31593322753906 -148.50775146484375 -148.40625 -148.6875
112.57125091552734 -148.2704315185547 -148.4630126953125 -148.28125 -148.46875
112.65499877929688 -148.29605102539062 -148.53240966796875 -148.28125 -148.46875
112.13624572753906 -148.43359375 -148.53314208984375 -148.1875 -148.53125
112.34624481201172 -148.3103485107422 -148.35531616210938 -148.40625 -148.375
112.1624984741211 -148.3035888671875 -148.3

112.10624694824219 -148.434326171875 -148.59759521484375 -148.65625 -148.15625
111.93999481201172 -148.51390075683594 -148.5991973876953 -148.28125 -148.78125
112.33499908447266 -148.5339813232422 -148.6256866455078 -148.40625 -148.8125
111.59124755859375 -148.62249755859375 -148.65032958984375 -148.65625 -148.53125
111.97750091552734 -148.67562866210938 -148.67041015625 -148.9375 -148.5625
111.79750061035156 -148.5398406982422 -148.5911102294922 -148.3125 -148.4375
112.54874420166016 -148.59288024902344 -148.59901428222656 -148.1875 -148.6875
111.99624633789062 -148.68319702148438 -148.6842498779297 -148.6875 -148.6875
112.60374450683594 -148.29554748535156 -148.58651733398438 -148.4375 -149.1875
112.35624694824219 -148.66163635253906 -148.7029266357422 -148.6875 -148.6875
111.58499908447266 -148.61561584472656 -148.61341857910156 -149.0625 -147.9375
112.07499694824219 -148.65086364746094 -148.65626525878906 -147.9375 -148.5625
112.52249908447266 -147.97640991210938 -148.5551147460937

112.26874542236328 -148.01023864746094 -148.4181671142578 -148.125 -148.59375
112.1937484741211 -148.23367309570312 -148.36273193359375 -148.40625 -148.46875
111.63874816894531 -148.37152099609375 -148.4915313720703 -148.84375 -148.65625
112.31624603271484 -148.27093505859375 -148.4833526611328 -148.84375 -148.40625
112.08499908447266 -148.474365234375 -148.4971160888672 -148.53125 -148.65625
111.59375 -148.5086669921875 -148.4779510498047 -148.78125 -148.71875
111.54000091552734 -148.34507751464844 -148.34767150878906 -148.34375 -148.34375
112.35250091552734 -148.2823028564453 -148.46304321289062 -148.84375 -148.21875
111.92499542236328 -148.29808044433594 -148.3603057861328 -148.34375 -148.40625
112.11249542236328 -148.12269592285156 -148.50172424316406 -147.59375 -148.65625
112.1875 -148.26156616210938 -148.38037109375 -148.15625 -148.3125
111.94749450683594 -148.37632751464844 -148.39877319335938 -148.375 -148.34375
112.11499786376953 -148.1856231689453 -148.46194458007812 -148.218

112.04374694824219 -148.5848388671875 -148.60877990722656 -148.53125 -148.90625
112.43999481201172 -148.21058654785156 -148.6048583984375 -148.6875 -148.40625
111.63874816894531 -148.65890502929688 -148.65997314453125 -148.625 -148.65625
112.04750061035156 -148.53866577148438 -148.55690002441406 -148.15625 -148.28125
112.47750091552734 -148.0587921142578 -148.54954528808594 -147.28125 -148.28125
111.91500091552734 -148.599365234375 -148.6081085205078 -148.53125 -148.90625
111.92374420166016 -148.3476104736328 -148.35084533691406 -148.375 -148.34375
112.08124542236328 -147.9384765625 -148.4993438720703 -147.875 -148.21875
112.29624938964844 -148.3024139404297 -148.3896026611328 -148.125 -148.34375
111.69874572753906 -148.322998046875 -148.36387634277344 -148.46875 -148.46875
112.07249450683594 -148.3436279296875 -148.34312438964844 -148.40625 -148.34375
112.04499816894531 -148.0 -148.50332641601562 -148.0 -147.84375
112.37625122070312 -148.40785217285156 -148.49252319335938 -148.3125 -1

112.36000061035156 -148.5487823486328 -148.57772827148438 -148.5 -148.1875
112.40374755859375 -148.65625 -148.65618896484375 -148.65625 -148.65625
112.89624786376953 -148.64683532714844 -148.65118408203125 -148.65625 -148.65625
111.99749755859375 -147.77589416503906 -148.57484436035156 -147.4375 -148.5625
112.67250061035156 -148.68304443359375 -148.65988159179688 -148.90625 -148.53125
112.18374633789062 -148.64659118652344 -148.66490173339844 -148.25 -148.65625
112.60874938964844 -148.62147521972656 -148.63644409179688 -148.65625 -148.90625
111.98749542236328 -148.54730224609375 -148.59671020507812 -148.28125 -148.03125
112.45249938964844 -147.27003479003906 -148.4348602294922 -146.875 -149.21875
111.63624572753906 -148.55889892578125 -148.53688049316406 -148.90625 -148.53125
112.55500030517578 -148.3583221435547 -148.5266571044922 -148.65625 -148.40625
112.4749984741211 -148.5426483154297 -148.590576171875 -148.28125 -148.53125
112.25249481201172 -148.40660095214844 -148.5541076660156

In [2]:
NN.load_state_dict(torch.load('2D_50points_small_95'))

<All keys matched successfully>

In [3]:
evaluate(10, 1003, 1, 2)

NameError: name 'evaluate' is not defined

In [3]:
def evaluate(epochs = 30000, npoints = 13, batchsize = 100, nsamples = 2):
    NN.eval()
    with torch.no_grad():
        for i in range(epochs):
            env.reset(npoints, batchsize, nsamples)
            """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
            memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
            #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
            tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
            #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############


            NN.eval()
            _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
            NN.train()
            """
            clip logprob so doesn't reinforce things it already knows
            TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
            """
            
            greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
            
            print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
  

In [6]:
for i in range(10):
    random_insertion(53, 1000)

112.23300170898438 7.801767826080322
112.27400207519531 8.104974746704102
112.65799713134766 7.472407341003418
112.23100280761719 7.915952205657959
112.4520034790039 7.713442325592041
111.92400360107422 8.143373489379883
112.55699920654297 8.011175155639648
112.1729965209961 8.002193450927734
112.03600311279297 7.991656303405762
111.97899627685547 7.911330223083496
